In [18]:
import numpy as np
import pandas as pd
from glob import glob
from lasagne import layers
from lasagne.updates import nesterov_momentum
from nolearn.lasagne import NeuralNet, BatchIterator
import theano
from theano.tensor.nnet import sigmoid
from sklearn.preprocessing import StandardScaler
from lasagne.objectives import aggregate, binary_crossentropy
import lasagne

In [40]:
def load_data():
    path="data/"
    data=pd.read_csv(path+"input_train.csv", index_col="ID").filter(regex= "EEG.*")
    labels=pd.read_csv(path+"challenge_output_data_training_file_sleep_stages_classification.csv", sep=";")

    # Load the dataset
    from sklearn.cross_validation import train_test_split
    return train_test_split(data.values, labels["TARGET"].values, test_size=0.10, random_state=42)
    

In [46]:
X_train2, X_test, y_train, y_test  = load_data()

In [65]:
X_train = X_train2.reshape(X_train2.shape[0],X_train2.shape[1],1)

In [66]:
def loss(x, t):
    return lasagne.objectives.categorical_crossentropy(x, t)


In [69]:
hidden_layer_size = 100  #change to 1024
N_EVENTS = 5
max_epochs = 20
NO_TIME_POINTS = 3750
nn = NeuralNet(
    layers=[
        ('input', layers.InputLayer),
        ('dropout1', layers.DropoutLayer),
        ('conv1', layers.Conv1DLayer),
        ('conv2', layers.Conv1DLayer),
        ('pool1', layers.MaxPool1DLayer),
        ('dropout2', layers.DropoutLayer),
        ('hidden4', layers.DenseLayer),
        ('dropout3', layers.DropoutLayer),
        ('hidden5', layers.DenseLayer),
        ('dropout4', layers.DropoutLayer),
        ('output', layers.DenseLayer),
    ],
    input_shape=(None, None)#NO_TIME_POINTS),
    dropout1_p=0.5,
    conv1_num_filters=4, conv1_filter_size=1,
    conv2_num_filters=8, conv2_filter_size=4, pool1_pool_size=4,
    dropout2_p=0.5, hidden4_num_units=hidden_layer_size,
    dropout3_p=0.5, hidden5_num_units=hidden_layer_size,
    dropout4_p=0.5, output_num_units=N_EVENTS, output_nonlinearity=sigmoid,

    batch_iterator_train = BatchIterator(batch_size=1000),
    batch_iterator_test = BatchIterator(batch_size=1000),

    y_tensor_type=theano.tensor.matrix,
    update=nesterov_momentum,
    update_learning_rate=theano.shared(0.03),
    update_momentum=theano.shared(0.9),

    objective_loss_function=loss,
    regression=True,

    max_epochs=max_epochs,
    verbose=1,
)

SyntaxError: invalid syntax (<ipython-input-69-431f2ace42db>, line 20)

In [68]:
nn.fit(X_train, y_train)


TypeError: Failed to instantiate <class 'lasagne.layers.pool.MaxPool1DLayer'> with args {'incoming': <lasagne.layers.conv.Conv1DLayer object at 0x7fad37dde990>, 'name': 'pool1'}.
Maybe parameter names have changed?

In [ ]:
predicted_label = nn.predict(X_test)
print("Random Forest - accuracy Score on test_data : ", accuracy_score(y_test, predicted_label))
print("Random Forest - kappa Score on test_data : ", cohen_kappa_score(y_test, predicted_label))
print("Random Forest - kappa Score on train data : ", cohen_kappa_score(y_train, rf.predict(X_train)))